In [26]:
import os
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# To get reproducible results
np.random.seed(0)
tf.set_random_seed(0)

print(os.listdir("../input"))

['movie-review-sentiment-analysis-kernels-only', 'fasttext-crawl-300d-2m']


In [27]:
test_file = '../input/movie-review-sentiment-analysis-kernels-only/test.tsv'
test = pd.read_csv(test_file, delimiter='\t').fillna('')
x_test = test.values[:, 2]

train_file = '../input/movie-review-sentiment-analysis-kernels-only/train.tsv'
train = pd.read_csv(train_file, delimiter='\t').fillna('')
x_train = train.values[:, 2]
y_train = train.values[:, 3]

print('x_test count: {}'.format(len(x_test)))
print('x_train count: {}'.format(len(x_train)))
print('y_train count: {}'.format(len(y_train)))

x_test count: 66292
x_train count: 156060
y_train count: 156060


In [28]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_length = 60
max_features = 20000

x_all = []
x_all.extend(x_test)
x_all.extend(x_train)

tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(x_train)
x_test_seq = tk.texts_to_sequences(x_test)

np_x_train = pad_sequences(x_train_seq, maxlen=max_length,  padding='post')
np_x_test = pad_sequences(x_test_seq, maxlen=max_length,  padding='post')
np_y_train = to_categorical(y_train)

print ('np_x_train shape: {}'.format(np_x_train.shape))
print ('np_x_test shape: {}'.format(np_x_test.shape))
print ('np_y_train shape: {}'.format(np_y_train.shape))

np_x_train shape: (156060, 60)
np_x_test shape: (66292, 60)
np_y_train shape: (156060, 5)


In [29]:
import tqdm

word_dict = tk.word_index
embedding_dim = 300
embeddings_index = {}

with open('../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec', 'r') as f:
    lines = f.readlines()

for i in tqdm.tqdm(range(len(lines))):
    values = lines[i].rstrip().rsplit(' ')
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')

max_features = min(max_features, len(word_dict) + 1)
embedding_matrix = np.zeros((max_features, embedding_dim))

for word, i in word_dict.items():
    if i >= max_features:
        break

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print('Embedding matrix: {}'.format(embedding_matrix.shape))


 10%|█         | 200355/2000001 [00:13<01:58, 15220.67it/s]


 20%|█▉        | 398803/2000001 [00:26<01:46, 15074.74it/s]


 30%|██▉       | 597075/2000001 [00:39<01:44, 13426.69it/s]


 39%|███▉      | 775383/2000001 [00:52<01:19, 15501.20it/s]


 49%|████▊     | 974737/2000001 [01:05<01:07, 15124.38it/s]


...

 77%|███████▋  | 1543160/2000001 [01:44<00:30, 14805.31it/s]


 87%|████████▋ | 1737045/2000001 [01:57<00:17, 15097.09it/s]


 95%|█████████▌| 1907698/2000001 [02:10<00:06, 14683.32it/s]


100%|██████████| 2000001/2000001 [02:17<00:00, 14597.16it/s]

Embedding matrix: (19479, 300)


In [30]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping

def one_input_classifier(index, input_length, max_features, class_num, embedding_dim, embedding_matrix):
    inputs = Input(shape=(input_length,), name='input_1')
    embeddings = Embedding(max_features, embedding_dim,
                           weights=[embedding_matrix], input_length=input_length,
                           trainable=False, name='embedding_1')(inputs)
    x = SpatialDropout1D(0.3, name='spatial_dropout1d_1')(embeddings)

    x = Bidirectional(CuDNNLSTM(128, name='lstm_1', return_sequences=True), name='bidirectional_1')(x)
    x = Dropout(0.25, name='dropout_1')(x)
    x = Conv1D(128, 5, activation='relu', name='conv1d_1')(x)
    x = Conv1D(128, 3, activation='relu', name='conv1d_2')(x)
    x = Conv1D(128, 1, activation='relu', name='conv1d_3')(x)
    x = Dropout(0.25, name='dropout_2')(x)

    x = GlobalMaxPooling1D(name='global_maxpool1d_1')(x)
    x = Dense(32, activation='relu', name='dense_1')(x)
    x = Dropout(0.25, name='dropout_5')(x)
    preds = Dense(class_num, activation='softmax', name='preds')(x)

    model = Model(inputs=inputs, outputs=preds, name='model_{}'.format(index))
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1,
                               mode='min', baseline=None, restore_best_weights=True)

class_num = np_y_train.shape[1]
epochs = 32
batch_size = 1024
validation_split = 0.2
classifier_num = 10

print('Classes: {}'.format(class_num))
print('Epochs: {}'.format(epochs))
print('Batch size: {}'.format(batch_size))
print('Validation split: {:.1}'.format(validation_split))
print('Classifiers: {}'.format(classifier_num))

Classes: 5
Epochs: 32
Batch size: 1024
Validation split: 0.2
Classifiers: 10


In [31]:
classifiers = []

for i in range(classifier_num):
    classifier = one_input_classifier(i, max_length, max_features, class_num, embedding_dim, embedding_matrix)

    if i == 0:
        classifier.summary()

    hist = classifier.fit(np_x_train, np_y_train, validation_split=validation_split, shuffle=True,
                          callbacks=[early_stopping], epochs=epochs, batch_size=batch_size, verbose=1)
    classifier.trainable = False
    classifiers.append(classifier)

    print('min loss ({}): {:.4}'.format(i, min(hist.history['loss'])))
    print('min val_loss ({}): {:.4}'.format(i, min(hist.history['val_loss'])))
    print('max acc ({}): {:.4}'.format(i, max(hist.history['acc'])))
    print('max val_acc ({}): {:.4}'.format(i, max(hist.history['val_acc'])))

y_pred_list = []

for i in range(classifier_num):
    y_pred = classifiers[i].predict(np_x_test, batch_size=1024, verbose=1)
    y_pred_list.append(y_pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 60, 300)           5843700   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 60, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 256)           440320    
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 56, 128)           163968    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 54, 128)           49280     
__________

124848/124848 [==============================] - 9s 74us/step - loss: 0.8020 - acc: 0.6695 - val_loss: 0.9005 - val_acc: 0.6309
Epoch 7/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7789 - acc: 0.6778 - val_loss: 0.8808 - val_acc: 0.6347
Epoch 8/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7624 - acc: 0.6859 - val_loss: 0.8616 - val_acc: 0.6407
Epoch 9/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7440 - acc: 0.6931 - val_loss: 0.8607 - val_acc: 0.6423
Epoch 10/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7327 - acc: 0.6970 - val_loss: 0.8650 - val_acc: 0.6413
Epoch 11/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7183 - acc: 0.7023 - val_loss: 0.8688 - val_acc: 0.6405
Restoring model weights from the end of the best epoch
Epoch 00011: early stopping
min loss (4): 0.7183
min val_loss (4): 0.8607
max acc (4): 0.7023
max val_acc (4): 0.6423
Tr

Epoch 4/32
124848/124848 [==============================] - 9s 75us/step - loss: 0.8819 - acc: 0.6373 - val_loss: 0.9099 - val_acc: 0.6233
Epoch 5/32
124848/124848 [==============================] - 9s 75us/step - loss: 0.8549 - acc: 0.6478 - val_loss: 0.8863 - val_acc: 0.6301
Epoch 6/32
124848/124848 [==============================] - 9s 75us/step - loss: 0.8263 - acc: 0.6610 - val_loss: 0.8934 - val_acc: 0.6259
Epoch 7/32
124848/124848 [==============================] - 9s 75us/step - loss: 0.8070 - acc: 0.6687 - val_loss: 0.8732 - val_acc: 0.6391
Epoch 8/32
124848/124848 [==============================] - 9s 75us/step - loss: 0.7880 - acc: 0.6761 - val_loss: 0.8764 - val_acc: 0.6378
Epoch 9/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7686 - acc: 0.6853 - val_loss: 0.8731 - val_acc: 0.6381
Epoch 10/32
124848/124848 [==============================] - 9s 74us/step - loss: 0.7524 - acc: 0.6903 - val_loss: 0.8661 - val_acc: 0.6390
Epoch 11/32
124848/124848 

In [86]:
test_num = np_x_test.shape[0]
y_pred_class = np.ndarray(shape=(test_num,), dtype=np.int32)

for i in range(test_num):
    votes = []

    for j in range(classifier_num):
        vote = y_pred_list[j][i].argmax(axis=0).astype(int)
        votes.append(vote)

    vote_final = max(set(votes), key=votes.count)
    y_pred_class[i] = vote_final

mapping = {phrase: sentiment for _, _, phrase, sentiment in train.values}

# Overlapping
for i, phrase in enumerate(test.Phrase.values):
    if phrase in mapping:
        y_pred_class[i] = mapping[phrase]

test['Sentiment'] = y_pred_class
test[['PhraseId', 'Sentiment']].to_csv('submission.csv', index=False)
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,3
1,156062,8545,An intermittently pleasing but mostly routine ...,3
2,156063,8545,An,2
3,156064,8545,intermittently pleasing but mostly routine effort,2
4,156065,8545,intermittently pleasing but mostly routine,3
